In [1]:
import requests, lxml.html
from bs4 import BeautifulSoup
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import datetime
import pickle

In [2]:
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

chromedriver = "/usr/lib/chromium-browser/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver) #chrome_options=options
driver.get('https://www.goodreads.com/user/sign_in')

In [3]:
username_form= driver.find_element_by_id("user_email")
username_form.send_keys("mburnamfink@gmail.com")

username_form= driver.find_element_by_id("user_password")
username_form.send_keys("mad5ki11z")
username_form.send_keys(Keys.RETURN)

In [4]:
#['  Bitten by Witch Fever: Wallpaper & Arsenic in the Victorian Home', 
#'Hawksley, Lucinda', 
#'4.27', 
#'137', 
#'Oct 2016', 
#'really liked it', 
#'0', 
#'1', 
#'Aug 07, 2018', 
#'Aug 07, 2018', 
#'view']

def shelf_scraper(shelf_name, driver):
    books = driver.find_elements_by_class_name('bookalike')
    
    rating_dict = {'did not like it': 1,
          'it was ok': 2,
          'liked it': 3,
          'really liked it': 4,
          'it was amazing': 5,
          '1 of 5 stars': 0}
    
    for book in books:
        lbook = book.text.split('\n')
        title =  lbook[0].strip()
        author = lbook[1].replace('*','').strip()
        avg_rating = lbook[2]
        times_rated = lbook[3]
        rating_text = lbook[5]
        try:
            rating =  rating_dict[rating_text]
        except:
            rating = rating_text
        comments = lbook[6]
        likes = lbook[7]
        date_added = lbook[-2]
        raw = book.get_attribute('innerHTML').strip()
        try:
            isbn_start = raw.find('isbn')
            isbn = raw[isbn_start+70:isbn_start+80]
            soup = BeautifulSoup(raw, 'lxml')
        except:
            isbn = ''
        review_url = soup.find('a', class_='nobreak').get('href') 
        a= (user, shelf_name, review_url, title, author, avg_rating, times_rated, rating, comments, likes, date_added, isbn)
        print(a)
        results.append(a)

In [5]:
#os.chdir(..)
#os.chdir('metis_03-McNulty')
os.getcwd()

'/home/michael/OneDrive/ds/metis/bootcamp/projects/metis_03-McNulty'

In [6]:
df = pd.read_csv('userids.csv', index_col=None, sep ='\t')
df.head()
userids =  list(df.userid.values)

In [7]:
os.chdir('scrapes')
os.getcwd()

'/home/michael/OneDrive/ds/metis/bootcamp/projects/metis_03-McNulty/scrapes'

In [8]:
jar = []

for file in os.listdir():
    if file.split(' ')[-1]=='shelf.pkl':
        jar.append(int(file.split(' ')[0]))

print(len(userids))
for file in jar:
    try:
        userids.remove(file)
    except:
        pass
print(len(userids))

print(userids[0], userids[-1])
userids = userids[::-1]
print(userids[0], userids[-1])

37786
34618
45618 7751086
7751086 45618


In [ ]:
for user in userids:
    try:
        t0 = time.time()
        user= str(user)
        user_url = 'https://www.goodreads.com/review/list/'+user

        driver.get(user_url)

        possibles = driver.find_elements_by_class_name('actionLinkLite')

        #//*[@id="paginatedShelfList"]/div[8]/a[2]
        #//*[@id="shelves"]/div[1]/a[1]
        #//*[@id="shelves"]/div[2]/a[7]

        shelves = []

        for item in possibles:
            try:
                text = item.text.split(' ')[0]
                link = item.get_attribute('href')
                if text == 'Read' or text == 'Currently' or text == 'Want':
                    pass
                else:
                    if 'shelf' in link:
                        print  (link, text)
                        shelves.append((link, text))
            except:
                print(i, 'ERROR')

        results = []
        for shelf in shelves:
            shelf_url = shelf[0]
            shelf_name =  shelf[1]
            driver.get(shelf_url)

            settings = driver.find_element_by_id('shelfSettingsLink')
            settings.click()
            time.sleep(.1)
            list_button = driver.find_element_by_id('listFieldSetLink')
            list_button.click()
            position_button = driver.find_element_by_id('position_field')
            if position_button.is_selected():
                position_button.click()

            while True:
                t1 =time.time()
                time.sleep(.750)
                shelf_scraper(shelf_name, driver)
                print('')
                print (datetime.datetime.now())
                print('%d books %.3f seconds' % (len(results), time.time()-t1), shelf_name)
                print('')
                try:
                    nextpage = driver.find_element_by_class_name('next_page')
                except:
                    break
                if nextpage.get_attribute('href') == None:
                    break
                else:
                    try:
                        nextpage.click()
                    except:
                        break

        filename = user+' shelf.pkl'
        outfile = open(filename,'wb')
        pickle.dump(results,outfile)
        outfile.close()

        print ('%d TOTAL TIME: %.3f' % (len(results), time.time()-t0))
    except:
        print('ERROR', user)
        pass

https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=dnf dnf
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=series-now-dead-to-me series-now-dead-to-me
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=wouldn-t-read-if-you-paid-me wouldn-t-read-if-you-paid-me
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=50soglike 50soglike
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=_arc-reads _arc-reads
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=_beta-reads _beta-reads
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=_to-be-finished _to-be-finished
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=action-adventure action-adventure
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=actor-heroes actor-heroes
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=actress-heroine actress-heroine
https://www.goodreads.com/review/list/7751086-ivy-deluca?shelf=aliens-

('7751086', 'dnf', '/review/show/2084236306', 'Dating the Undead (Bite Nights, #1)', 'Lyons, Juliet', '3.68', '811', '0', '1', 'not set', 'Aug 06, 2017', 'iv></td>  ')
('7751086', 'dnf', '/review/show/2083663030', 'The Red Scot (A Curvy Girls Club Novel, #1)', 'Turner, Twyla', '4.04', '663', '', '', '3', 'Aug 05, 2017', 'iv></td>  ')
('7751086', 'dnf', '/review/show/1387689596', "The Lies Between Us (The Devil's Dust, #4)", 'Forgy, M.N.', '4.37', '1,758', '0', '1', 'Sep 14, 2015', 'Sep 09, 2015', 'iv></td>  ')
('7751086', 'dnf', '/review/show/1355632710', 'Hate to Love You (Love/Hate, #1)', 'Richards, Isabelle', '4.03', '866', '0', '1', 'Sep 06, 2015', 'Aug 05, 2015', 'iv></td>  ')
('7751086', 'dnf', '/review/show/1309517741', 'Practice Makes Perfect (The Warriors, #3)', 'Groome, Charlene', '2.72', '53', '0', '1', 'Aug 12, 2015', 'Jun 16, 2015', 'iv></td>  ')
('7751086', 'dnf', '/review/show/1251145608', 'Strike: Dax (Sphere of Irony #2)', 'Heather C. Leigh', '4.12', '218', '', '', '2'

('7751086', 'wouldn-t-read-if-you-paid-me', '/review/show/2383084638', 'Cocky Roomie (Cocker Brothers of Atlanta #1)', 'Hopkins, Faleena', '3.73', '5,295', '0', '6', 'not set', 'May 05, 2018', 'iv></td>  ')
('7751086', 'wouldn-t-read-if-you-paid-me', '/review/show/2281030738', 'Good Luck with That', 'Higgins, Kristan', '4.18', '3,090', '0', '0', 'not set', 'Feb 01, 2018', '045148939X')
('7751086', 'wouldn-t-read-if-you-paid-me', '/review/show/1322579419', 'Go Set a Watchman (To Kill a Mockingbird, #2)', 'Lee, Harper', '3.31', '191,669', '', '', '', 'Jul 01, 2015', '0062409859')
('7751086', 'wouldn-t-read-if-you-paid-me', '/review/show/1272550230', 'Beautiful Sacrifice (The Maddox Brothers, #3)', 'McGuire, Jamie', '4.08', '25,036', '0', '0', 'not set', 'May 05, 2015', 'iv></td>  ')
('7751086', 'wouldn-t-read-if-you-paid-me', '/review/show/1234096628', 'Black Balled', 'Smith, Andrea', '3.39', '1,240', '0', '1', 'not set', 'Mar 22, 2015', 'iv></td>  ')
('7751086', 'wouldn-t-read-if-you-pa

('7751086', '_arc-reads', '/review/show/2483466858', 'Unconditionally Mine', 'Gonzalez, Nadine', '4.27', '22', 4, '', '', 'Aug 05, 2018', '1335216812')
('7751086', '_arc-reads', '/review/show/2480196189', 'Boy Toy (Man Hands, #3)', 'Bowen, Sarina', '4.21', '2,055', 4, '', '', 'Aug 02, 2018', 'iv></td>  ')
('7751086', '_arc-reads', '/review/show/2474322681', 'It Seemed Like a Good Idea at the Time', 'Scott, Kylie', '3.75', '2,557', 4, '', '', 'Jul 29, 2018', 'iv></td>  ')
('7751086', '_arc-reads', '/review/show/2473355117', 'A Ring for Rosie', 'Wells, Maggie', '3.43', '21', 2, '', '', 'Jul 28, 2018', 'iv></td>  ')
('7751086', '_arc-reads', '/review/show/2466460525', 'Down with Love (Laws of Attraction, #1)', 'Meader, Kate', '4.01', '641', 4, '', '', 'Jul 22, 2018', 'iv></td>  ')
('7751086', '_arc-reads', '/review/show/2445711901', 'Playing Dirty (Sydney Smoke Rugby, #6)', 'Andrews, Amy', '4.26', '160', 4, '', '', 'Jul 04, 2018', '1640635491')
('7751086', '_arc-reads', '/review/show/2442

('7751086', '_to-be-finished', '/review/show/1904824204', 'Heat', 'Smith, R. Lee', '3.82', '1,714', '0', '0', 'not set', 'Feb 06, 2017', 'iv></td>  ')
('7751086', '_to-be-finished', '/review/show/1385961677', 'Fearless (Dartmoor, #1)', 'Gilley, Lauren', '4.29', '3,558', '0', '2', 'not set', 'Sep 07, 2015', '1505588391')

2018-10-23 16:41:15.671415
144 books 0.949 seconds _to-be-finished

('7751086', '_to-be-finished', '/review/show/1904824204', 'Heat', 'Smith, R. Lee', '3.82', '1,714', '0', '0', 'not set', 'Feb 06, 2017', 'iv></td>  ')
('7751086', '_to-be-finished', '/review/show/1385961677', 'Fearless (Dartmoor, #1)', 'Gilley, Lauren', '4.29', '3,558', '0', '2', 'not set', 'Sep 07, 2015', '1505588391')

2018-10-23 16:41:16.672261
146 books 0.856 seconds _to-be-finished

('7751086', '_to-be-finished', '/review/show/1904824204', 'Heat', 'Smith, R. Lee', '3.82', '1,714', '0', '0', 'not set', 'Feb 06, 2017', 'iv></td>  ')
('7751086', '_to-be-finished', '/review/show/1385961677', 'Fearless

...

('2172094', '2016', '/review/show/1404621155', 'The Sword of Summer (Magnus Chase and the Gods of Asgard, #1)', 'Riordan, Rick', '4.26', '114,577', 4, '0', '0', 'Sep 30, 2015', '1423160916')
('2172094', '2016', '/review/show/1341359059', 'The Raven King (The Raven Cycle, #4)', 'Stiefvater, Maggie', '4.31', '66,282', 4, '', '', 'Jul 21, 2015', '0545424984')
('2172094', '2016', '/review/show/1322485357', 'Bitch Planet, Vol. 1: Extraordinary Machine', 'DeConnick, Kelly Sue', '4.05', '18,872', 3, '0', '0', 'Jul 01, 2015', '1632153661')
('2172094', '2016', '/review/show/1316342879', 'Hawkeye, Volume 3: L.A. Woman', 'Fraction, Matt', '4.15', '8,889', 4, '0', '0', 'Jun 24, 2015', '0785183906')
('2172094', '2016', '/review/show/1196469975', 'Trouble Is a Friend of Mine (Trouble, #1)', 'Tromly, Stephanie', '3.98', '4,424', 5, '0', '1', 'Feb 10, 2015', '0525428402')
('2172094', '2016', '/review/show/1028752310', 'This One Summer', 'Tamaki, Mariko', '3.66', '43,770', 3, '', '', 'Aug 16, 2014', '1

('2172094', '2017', '/review/show/1966563641', 'The Woman in Cabin 10', 'Ware, Ruth', '3.68', '220,557', 2, '0', '2', 'Apr 09, 2017', '1501132938')
('2172094', '2017', '/review/show/1957097963', 'Still Life (Chief Inspector Armand Gamache, #1)', 'Penny, Louise', '3.88', '100,121', 4, '0', '1', 'Mar 30, 2017', '0312948557')
('2172094', '2017', '/review/show/1951881738', 'The Little Book of Hygge: The Danish Way to Live Well', 'Wiking, Meik', '3.72', '35,625', 5, '0', '1', 'Mar 25, 2017', '0241283914')
('2172094', '2017', '/review/show/1946535476', "The Assassin's Curse (The Blackthorn Key, #3)", 'Sands, Kevin', '4.47', '885', 5, '0', '0', 'Mar 19, 2017', '1534405232')
('2172094', '2017', '/review/show/1931755551', 'The Swan Riders (Prisoners of Peace, #2)', 'Bow, Erin', '3.85', '1,029', 4, '0', '0', 'Mar 04, 2017', 'iv></td>  ')
('2172094', '2017', '/review/show/1929499587', 'Frogkisser!', 'Nix, Garth', '3.79', '3,083', 4, '0', '0', 'Mar 02, 2017', '1760293512')
('2172094', '2017', '/re

('2172094', '2018', '/review/show/2393407200', 'Obama: An Intimate Portrait, Deluxe Limited Edition: The Historic Presidency in Photographs', 'Souza, Pete', '4.17', '7,879', 5, '0', '0', 'May 16, 2018', '0316512591')
('2172094', '2018', '/review/show/2392381185', 'Honor Among Thieves (The Honors, #1)', 'Caine, Rachel', '3.89', '1,833', 4, '0', '2', 'May 15, 2018', 'iv></td>  ')
('2172094', '2018', '/review/show/2386708906', 'Lady Knight (Protector of the Small, #4)', 'Pierce, Tamora', '4.31', '43,147', 4, '', '', 'May 09, 2018', 'iv></td>  ')
('2172094', '2018', '/review/show/2384716100', "Trickster's Choice (Daughter of the Lioness, #1)", 'Pierce, Tamora', '4.28', '52,491', 4, '0', '0', 'May 07, 2018', 'iv></td>  ')
('2172094', '2018', '/review/show/2380049812', 'Squire (Protector of the Small, #3)', 'Pierce, Tamora', '4.28', '43,191', 5, '0', '1', 'May 02, 2018', '0375829067')
('2172094', '2018', '/review/show/2365304213', 'Page (Protector of the Small, #2)', 'Pierce, Tamora', '4.23'

('2172094', 'arc', '/review/show/1404621492', 'An Inheritance of Ashes', 'Bobet, Leah', '3.56', '1,034', 3, '0', '0', 'Sep 30, 2015', '054428111X')
('2172094', 'arc', '/review/show/1374015220', 'The Cage (The Cage, #1)', 'Shepherd, Megan', '3.64', '5,387', 2, '0', '0', 'Aug 25, 2015', 'iv></td>  ')
('2172094', 'arc', '/review/show/1242750419', '99 Days', 'Cotugno, Katie', '3.38', '14,769', 3, '0', '0', 'Apr 01, 2015', '0062216384')
('2172094', 'arc', '/review/show/1201854779', 'Red Queen (Red Queen, #1)', 'Aveyard, Victoria', '4.07', '403,538', 3, '0', '0', 'Feb 15, 2015', '1409151174')
('2172094', 'arc', '/review/show/1151008211', 'The Darkest Part of the Forest', 'Black, Holly', '3.91', '34,367', 4, '0', '0', 'Jan 02, 2015', '0316213071')
('2172094', 'arc', '/review/show/1113412296', 'The Sin Eater’s Daughter (The Sin Eater’s Daughter, #1)', 'Salisbury, Melinda', '3.47', '13,000', 2, '', '', 'Nov 23, 2014', '1760151572')
('2172094', 'arc', '/review/show/983158916', 'The Iron Trial (M

('2172094', 'arc', '/review/show/135865113', 'Wither (The Chemical Garden, #1)', 'DeStefano, Lauren', '3.81', '93,359', 2, '', '', 'Dec 20, 2010', '1442409053')
('2172094', 'arc', '/review/show/131138825', 'Delirium (Delirium, #1)', 'Oliver, Lauren', '3.99', '383,761', 4, '0', '0', 'Nov 16, 2010', '0061726826')

2018-10-23 22:38:47.167242
724 books 6.952 seconds arc

('2172094', 'art', '/review/show/50988469', "Arthur Spiderwick's Field Guide to the Fantastical World Around You", 'DiTerlizzi, Tony', '4.34', '5,242', 4, '0', '0', 'Mar 30, 2009', '0689859414')
('2172094', 'art', '/review/show/50988076', 'Out of Picture Volume 2: Art from the Outside Looking In', 'OOP, The Artists of', '4.00', '42', 4, '0', '0', 'Mar 30, 2009', '0345498739')
('2172094', 'art', '/review/show/50899797', 'Klimt', 'Costantino, Maria', '4.38', '26', '0', '0', 'not set', 'Mar 30, 2009', '1577150538')
('2172094', 'art', '/review/show/50899765', 'History of Modern Art: Painting Sculpture Architecture Photography'